In [ ]:
!pip3 install git+https://github.com/Centre-automatique-et-systemes/lena.git gwpy &> /dev/null
!pip3 install git+https://github.com/aliutkus/torchinterp1d.git gwpy &> /dev/null

In [ ]:
import sys ; sys.path.append('../')
from lena.luenberger_observer import LuenbergerObserver
from lena.system import RevDuffing
from lena.learner import Learner

In [ ]:
# Set up the system
system = RevDuffing()

In [ ]:
# Generate the data
T = LuenbergerObserver(dim_x=2, dim_y=1, method="Autoencoder", wc=1.5)
T.set_dynamics(system)
data = T.generate_data_svl([2.0,2.0], 20000)

learner_T = Learner(T, data, True, "T")
learner_T.train()

In [ ]:
T_star = LuenbergerObserver(2, 1, "T_star")
T_star.set_dynamics(system)

learner_T_star = Learner(T, data, True, "T_star")
learner_T_star.train()

In [ ]:
import torch 
tsim = (0, 20)
dt = 1e-2
tq, simulation = system.simulate(torch.tensor([[1.],[1.]]), tsim, dt)
measurement = T_star.h(
        simulation[:, :T_star.dim_x, 0].T).T
y = torch.cat((tq.unsqueeze(1), measurement), dim=1)

# Predict from measurment
estimation = T_star.predict(y, tsim, dt)

In [ ]:
import matplotlib.pyplot as plt

# Plot truth ground data and estimation
plt.plot(tq, simulation[:,:,0])
plt.plot(tq, estimation.detach().numpy())
plt.show()